In [3]:
from typing import Iterable, Tuple
import numpy as np
import pandas as pd
from IPython.display import Markdown, Latex

# Calculations go up here... These will be hidden in the final pdf...

# USEFUL FUNCTIONS AND VARIABLES:
def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return value * np.sqrt(total)

class UNIT:
    KG = ("kg", 1)
    G = ("g", 1000)
    M = ("m", 1)
    CM = ("cm", 100)
    MPS = (r"\frac {m}{s}", 1)
    RAD = ("rad", 1)

g = 9.8

def to_doc(val: float, val_err: float = None, unit: str = ("", 1), extra: str = "", dec_keep: int = 2, ds: int = 1) -> str:
    """ Converts values to a string for being displayed in the document. """
    s, mult = unit
    ds = "$" * ds
    if(val_err is not None):
        return fr"{ds}{extra}{val * mult:.0{dec_keep}f} \pm {val_err * mult:.0{dec_keep}f}\text{{ }}{s}{ds}"
    else:
        return fr"{ds}{extra}{val * mult:.0{dec_keep}f} \text{{ }}{s}{ds}"

def disp_table(df: pd.DataFrame) -> Latex:
    """ Displays a pandas table with index stripped off of it. """
    return Markdown(df.to_markdown(index=False))

def latex(*args: Iterable[str], dsc: int = 1) -> str:
    ds = "$" * dsc
    return f"{ds}{''.join(args)}{ds}"

def fstr(data: str, fmt: str) -> str:
    return f"{data:{fmt}}"

def sci(num: float, fmt: str) -> str:
    s = f"{num:{fmt}E}"
    num, power = s.split('E')
    return f"{num} \cdot 10^{{{power}}}"

In [4]:
# PART 1:
m = 65.5 / 1000 # g -> kg
M = 191.7 / 1000 # g -> kg
R = 27.7 / 100 # cm -> m

m_err = 0.1 / 1000 # g -> kg
M_err = 0.1 / 1000 # g -> kg
R_err = 0.2 / 100 # cm -> m

p1d = pd.read_csv("Part1Data.csv")

p1disp = pd.DataFrame(data=np.array([np.arange(1, len(p1d) + 1), p1d["Theta(deg)"]]).T, columns=["Trial", r"Theta $\theta^{\circ}$"])
p1disp["Trial"] = p1disp["Trial"].astype(int, copy=False)
theta_avg_rad = np.deg2rad(np.mean(p1d["Theta(deg)"]))
# The velocity in m/s...
v = ((m + M) / m) * np.sqrt(2 * g * R * (1 - np.cos(theta_avg_rad)))

cosVals = 1 - np.cos(np.deg2rad(p1d["Theta(deg)"]))

mM_err = np.sqrt(M_err ** 2 + m_err ** 2)
cos_val = np.mean(cosVals)
cos_err = np.std(cosVals) / np.sqrt(len(cosVals))

# Using the nice rule 4 function...
v_err = rule_4(v, (M + m, mM_err, 1), (m, m_err, -1), (R, R_err, 1/2), (cos_val, cos_err, 1/2))

In [5]:
# PART 2:
p2d = pd.read_csv("Part2Data.csv")

p2disp = pd.DataFrame(data=np.array([np.arange(1, len(p2d) + 1), p2d["Distance (cm)"]]).T, columns=["Trial", r"Distance (cm)"])
p2disp["Trial"] = p2disp["Trial"].astype(int, copy=False)

D1 = 193.7 / 100 # cm -> m
D2 = 18.1 / 100 # cm -> m

p2d["Total Dist(m)"] = D1 + D2 + (p2d["Distance (cm)"] / 100)

x = np.mean(p2d["Total Dist(m)"])
x_err = np.std(p2d["Total Dist(m)"]) / np.sqrt(len(p2d["Total Dist(m)"]))

y1 = 91.5 / 100 # cm -> m
y2 = 7.6 / 100 # cm -> m
yp_err = 0.1 / 100 # cm -> m
y = (y1 + y2)
y_err = np.sqrt(yp_err ** 2 + yp_err ** 2) 

v2 = x * np.sqrt(g / (2 * y))
v2_err = rule_4(v2, (y, y_err, -1/2), (x, x_err, 1))

## Abstract

The goal of this experiment was to compare 2 computational physical models to each other by computing the initial velocity of a projectile using both methods and comparing the result to check for consistency. The first method launched the projectile into a pendulum and used the conservation of energy and conservation of momentum laws to compute the initial velocity from the max angle the pendulum reached, while the second method used kinematics equations to find the initial velocity from the projectile's launch height and distance traveled. The results didn't equate, but it was concluded that this was likely due to underestimating the error of certain values in parts 1 and 2 of the experiment.

## Introduction

In this lab, the initial velocity of a projectile was calculated using two fundamental physical models, and the results were compared to see if the models agree with each other. The two models that were used to compute the ball's initial velocity in this experiment were the conservation of energy laws and kinematics equations.

### Method 1

First, the velocity of the projectile was calculated using the conservation of momentum and the conservation of energy laws. To do this, the projectile(a round metal ball) was launched into a pendulum, which captured the ball. Then the max angle of the pendulum was taken as it swung up, as shown in figure 1. 
![Image of the model for part 1](Part1Diagram.png)
The collision between the pendulum and the ball is inelastic, and therefore the momentum before the collision must be maintained after the collision. If the mass of the ball is defined as $m$, the velocity of the ball as $v$, the mass of the pendulum as $M$, and the velocity of the pendulum and ball combined as $V$, the relationship can be defined as $mv=(m + M)V$. Rearranging this formula provides a way to solve for the original ball's velocity.

$$
v = \frac {(m + M)V}{m}
$$

By further observation of the system, it can be seen that the energy of the system is conserved, and is either stored in the form of kinetic or gravitational potential energy. Immediately after the collision, all energy is of kinetic form, and can be easily calculated using $KE = \frac{1}{2}(m + M)V^2$. When the pendulum reaches its maximum height or angle, the pendulum stops for a split second. Therefore, all energy is found in gravitational potential form and can be expressed as $PE=(m + M)g\Delta h$. Applying the conservation of energy law, it can be assumed the initial kinetic energy is equal to the final potential energy. This can be written as below, and rewritten to solve for the velocity of the pendulum.

$$
\frac{1}{2}(m + M)V^2 = (m + M)g\Delta h
$$
$$
V = \sqrt{2g \Delta h}
$$

It is possible to compute the height the pendulum lifts by the application of trigonometric functions. The height is the difference between the pendulum height and the final y component of the pendulum. If the length of the pendulum bob is defined as $R$, and the angle is defined as $\theta$, the change in the height of the pendulum can be expressed as below.

$$
\Delta h = R - R \cos(\theta) = R(1 - \cos(\theta))
$$

By combining equations $(1)$, $(2)$, and $(3)$, as shown above, the initial velocity of the ball can be computed from the pendulum height and angle using the formula shown below.

$$
v = \frac {m + M}{m} \sqrt {2gR(1 - cos(\theta))}
$$

### Method 2

For the second part, the velocity of the projectile was measured using kinematics equations. To accomplish this, the ball was launched from a table with a given height and the distance the ball traveled was measured, as shown in figure 2. 
![Image of the model for part 2](Part2Diagram.png)
Ignoring air resistance, given its impact on the system is minimal, there is no acceleration in the x-direction. The only acceleration which affects the system is the acceleration due to gravity, which only acts on the y component of the motion. This system can be represented using 2 kinematic equations, namely $\Delta x = v_i t$ and $\Delta y = \frac{1}{2}gt^2$, where $\Delta x$ is the displacement on the x-axis, $\Delta y$ is the displacement on the y-axis, and $v_i$ is the initial velocity. Solving this system for $v_i$ gives us the formula below for computing the initial velocity.
$$
v_i = \Delta x \sqrt{\frac {g}{2\Delta y}}
$$

## Data

### Method 1
For the first part of the experiment the length of the pendulum $R$, the mass of the pendulum $M$, and the mass of the ball $m$ were needed. The length of the pendulum from the pivot point to the center of mass was measured to be {{to_doc(R, R_err, UNIT.CM, r"R=")}}. The center of mass was determined by adjusting the pendulum until it balanced on a thin metal pole. The mass of the pendulum was measured on a digital scale to be {{to_doc(M, M_err, UNIT.G, r"M = ")}}. The mass of the ball, also measured on a digital scale, was determined to be {{to_doc(m, m_err, UNIT.G, r"m = ")}}. Once all preliminary measurements were taken, the ball was launched into the pendulum 10 times, and the max angle the pendulum reached was recorded. The resulting angles can be seen in table 1. 
{{disp_table(p1disp)}}

### Method 2

For the second part of the experiment, the ball was launched onto a piece of carbon paper placed on the ground, and the impact of the ball would leave a mark in the regular piece of paper placed underneath the ball. The distance from the edge of the paper to the table was measured to be {{f"{D1 * 100:.02f} cm"}}, and the distance from the edge of the table to the launcher was measured and recorded as {{f"{D2 * 100:.02f} cm"}}. Once these values were calculated, the ball was launched 11 times, and the distance from the edge of the paper to the ball impact was recorded. The resulting measurements can be seen in table 2.
{{disp_table(p2disp)}}


## Data Analysis and Results

### Method 1
After the data was collected, the average maximum swing angle was computed to be {{latex(r"\theta_{avg} =", fstr(theta_avg_rad, ".02f"), "\text{ rad}")}}. Once the average angle was computed, the average angle and the length of the pendulum were substituted into equation $(4)$.

$$
v = \frac {m + M}{m} \sqrt {2gR(1 - cos(\theta))} = \frac {m + M}{m} \sqrt {2gR(1 - cos(\theta_{avg}))}
$$
{{to_doc(v, None, UNIT.MPS, r"v = ", ds=2)}}

The uncertainty of the velocity still needed to be calculated. The formula for uncertainty was found by applying the rules of error propagation, specifically rule 4.

$$
\delta v = v \sqrt{(\frac{\delta(m + M)}{m + M})^2 + (-\frac{\delta m}{m})^2 + (\frac{\delta R}{2R})^2
+ (\frac{\delta(1 - \cos(\theta))}{2(1 - \cos(\theta))})^2}
$$

The above formula has several other errors that need to be calculated. To get $\delta(m + M)$, we can apply error propagation rule 2, which gives us $\delta(m + M) = \sqrt{\delta m^2 + \delta M^2}$. The uncertainty of $m$ and $M$ was {{to_doc(m_err, unit=UNIT.G)}} and {{to_doc(M_err, unit=UNIT.G)}}, respectively, as determined by the known uncertainty of the measuring instruments used. Plugging $m$ and $M$ into the formula above gave a result of
{{to_doc(mM_err, unit=UNIT.G)}}. To get $\delta(1 - \cos(\theta))$, every angle was run through $1 - \cos(\theta)$, and the standard deviation of all of these values was taken and divided by the number of trials. This formula takes the form below.
$$
\delta(1 - \cos(\theta)) = \frac {\sigma}{\sqrt{n}} = \frac {\sigma(1 - \cos(\theta))}{\sqrt{n}}
$$
Where the standard deviation is.
$$
\sigma(X)=\sqrt{\frac{1}{N-1}\sum_{i=1}^N(X_i - X_{avg})^2}
$$

Applying this formula results in {{latex(r"\delta(1 - \cos(\theta)) = ", sci(cos_err, ".02"))}}. Finally, knowing {{to_doc(R_err, unit=UNIT.CM, extra=r"\delta R = ")}} due to uncertainty in the measuring tool used, we can solve for the uncertainty in velocity. This gives an initial velocity of {{to_doc(v, v_err, UNIT.MPS, "v_i = ")}} for part one of the experiment.

### Method 2

After data was collected for part 2, the lengths were added to the distance from the launcher to the edge of the table and the distance from the edge of the table to the paper to get the total x displacement for each ball launch. The average x displacement was then computed to be {{to_doc(x, None, UNIT.CM)}}. The error in the x displacement was computed by taking the standard deviation in the x displacement and dividing it by the square root of the number of trails, as shown in equations $(8)$ and $(9)$. After computing the error, the x displacement is {{to_doc(x, x_err, UNIT.CM, "x = ")}}. The y displacement was measured in 2 sections, and after adding them and applying error propagation rule 2, the result {{to_doc(y, y_err, UNIT.CM, "y = ")}} is found. The initial velocity can now be computed by applying equation $(5)$ and also applying the formula below for finding the error in velocity, derived using error propagation rule 4. 

$$
\delta v = v \sqrt{(\frac{-\delta y}{2y})^2 + (\frac{\delta x}{x})^2}
$$

After using equation $(5)$ and $(10)$, the initial velocity is found to be {{to_doc(v2, v2_err, UNIT.MPS, "v_i = ")}} for part 2. 

## Conclusions

The purpose of this experiment was to compute the initial velocity of a projectile (metal ball) using 2 different physical models and compare the results for consistency. For part 1 the conservation of energy and conservation of momentum laws were applied to the projectile as it entered a pendulum, and a experimental value of {{to_doc(v, v_err, UNIT.MPS, "v_i = ")}} was found for the velocity. For part 2, basic kinematics equations were applied to the projectile by retrieving its distance and height of launch, giving an experimental initial velocity of {{to_doc(v2, v2_err, UNIT.MPS, "v_i = ")}}. The results for parts 1 and 2 do not agree with each other as their uncertainty ranges don't overlap at any location, and therefore one could argue that the physical models are not equivalent. A more likely and valid conclusion is that the results didn't match due to underestimating the errors for both parts of the experiment. Specifically, in part 1 of the experiment, the error given for the pendulum length is only equal to the uncertainty of the actual measurement, when it might be possible to balance the pendulum through a range of values near the center of mass, which should have been included in the uncertainty. Also, in part 2 of the experiment, the errors in the measurements of the displacement between the table and paper and table and launcher were entirely ignored, which could contribute to a much higher uncertainty value in part. Also, the collision in part 1 of the experiment is truly not inelastic, as the collision generates sound, meaning some energy is lost. Finally, in part 2 the effect of air resistance is completely ignored, which could affect the result slightly. Given all of these possible areas for error to be introduced, and considering how close the values are, it would not be surprising to see the values showing correlation by fixing any of the issues listed above. The results of this experiment are rather inconclusive, and this experiment would need to be repeated to extract more information from the results.